## 简单的字符神经网络，给不同的姓氏分类

###  读取数据

In [1]:
import torch
from io import open
import os
import unicodedata
import string

In [2]:
all_letters = string.ascii_letters + ".,:''"
n_letters = len(all_letters)

In [3]:
def unicode2ascii(s):
    """
    转换unicode编码的字符为纯ascii字符串
    """
    return ''.join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


print(unicode2ascii('Ślusàrski'))

Slusarski


In [4]:
def lines2ascii(file):
    result = []
    for line in file.readlines():
        result.append(unicode2ascii(line).strip())
    return result


def read_data(root):
    """
    读取名字和语言数据为一个字典
    """
    result = {}
    for file_name in os.listdir(root):
        if str(file_name).endswith('.txt'):
            lan = file_name.replace('.txt', '')
            with open(os.path.join(root, file_name), encoding='UTF-8') as file:
                result[lan] = lines2ascii(file)
    return result


data = read_data('./data/names')

In [5]:
all_categories = list(data.keys())
n_category = len(all_categories)

### 将字符数据映射为向量数据，使用one hot向量表示

In [6]:
def char2Vec(c):
    """
    映射字符为向量数据，输入为字符，输出为宽度与 all_letters长度相同的向量
    """
    vec = torch.zeros(len(all_letters))
    vec[all_letters.index(c)] = 1
    return vec
print(char2Vec('d'))

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.])


In [7]:
def str2Vec(name):
    """
    将字符串转换为vec表示，形状为 len*1*n_letter
    """
    vec = torch.zeros(len(name), 1, len(all_letters))
    for i in range(len(name)):
        vec[i][0][all_letters.index(name[i])] = 1
    return vec


print(str2Vec('ab'))

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]]])


### 构建RNN

In [8]:
import torch.nn as nn

In [9]:
class Rnn(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        自定义rnn运行所需的函数
        """
        super(Rnn, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, data, hidden):
        """
        rnn运行的数据流，如何使用上述函数
        """
        combine = torch.cat((data, hidden), 1)
        new_hidden = self.i2h(combine)
        output = self.i2o(combine)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.randn(1, self.hidden_size)
        

n_hidden = 1024
rnn = Rnn(n_letters, n_hidden, n_category)

> 测试rnn运行单步


In [10]:
hidden = torch.randn(1, n_hidden)
name = "alex"
vec_name = str2Vec(name)
output, new_hidden = rnn(vec_name[0], hidden)
print(output)

tensor([[0.0354, 0.1391, 0.0160, 0.0385, 0.0603, 0.0461, 0.0382, 0.0864, 0.0418,
         0.0440, 0.0444, 0.0536, 0.0336, 0.0612, 0.0363, 0.0830, 0.1015, 0.0406]],
       grad_fn=<SoftmaxBackward>)


###  训练网络

In [11]:
def get_category(output):
    """
    获取最高概率的输出并转换为字符串
    """
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(get_category(output))


('German', 1)


In [12]:
import random

In [13]:
def random_chioce(data):
    """
    在列表中随机选择一个条目返回
    """
    return data[random.randint(0, len(data)-1)]

def get_vec_label(lan):
    """
    将标签从语言转换为one hot向量
    """
    vec = torch.tensor([all_categories.index(lan)], dtype=torch.long)
    return vec

def get_train_examples(train_data):
    """
    在训练数据中随机选择1训练条目
    """
   
    lan = random_chioce(all_categories)
    name = random_chioce(train_data[lan])
    lan_vec = get_vec_label(lan)
    name_vec = str2Vec(name)
   
    return name_vec, lan_vec

print(get_train_examples(data))
        

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 

In [18]:
loss_func = nn.NLLLoss()
lr = 0.005

def train_step(name_vec, lan_vec):
    """
    单步训练，连续输入一串字符，更新梯度，并计算损失
    """
    hidden = rnn.init_hidden()
    "梯度清零"
    rnn.zero_grad()
    
    "调用rnn预测"
    for i in range(name_vec.size()[0]):
        output, hidden = rnn(name_vec[i], hidden)
    
    "计算损失"
    loss = loss_func(output, lan_vec)
    "从损失函数开始反向传播梯度"
    loss.backward()
    
    "将rnn中的参数手动梯度下降"
    
    for p in rnn.parameters():
        if p.grad is not None:
            p.data.add_(-lr, p.grad.data)
    
    return output, loss.item()

In [19]:
import time

def train():
    """
    训练rnn，并保存loss
    """
    n_iters = 200000
    plot_every = 1000
    print_every = 5000
    
    all_losses = []
    cur_loss = 0
    start_time = time.time()
    for i in range(n_iters):
        "获取数据并预测，计算loss， 梯度下降"
        name_vec, lan_vec = get_train_examples(data)
        output, loss = train_step(name_vec, lan_vec)
        cur_loss += loss
        "打印"
        if i % print_every == 0:    
            print(time.time() - start_time)
            print("   ")
            print(loss)
        if i % plot_every == 0:
            all_losses.append(cur_loss / plot_every)
            cur_loss = 0

train()
        
        



0.0026199817657470703
   
-0.03318382054567337
4.865108013153076
   
-0.04594032093882561
9.823606014251709
   
-0.0737682580947876
15.028035879135132
   
-0.04835863038897514
19.969321966171265
   
-0.12216559797525406
24.751803874969482
   
-0.07109123468399048
29.627595901489258
   
-0.05699862539768219
34.503588914871216
   
-0.060935478657484055
39.388895988464355
   
-0.043434321880340576
44.161267042160034
   
-0.03135152906179428
48.977444887161255
   
-0.07290792465209961


KeyboardInterrupt: 